In [1]:
import os
import sys

import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random

In [2]:
root = os.path.dirname(os.getcwd())
sys.path.append(root)

df = pd.read_csv(root + os.sep + 'data'+ os.sep + 'BASE.csv')

In [3]:
from src.utils.models import CharacterPreprocessor

In [4]:
cp = CharacterPreprocessor(df)
cp.preprocess()

Corpus length: 630844
Total chars: 52
Number of sequences: 210268


In [ ]:
#baseline = keras.models.load_model(root + os.sep + 'models'+ os.sep + 'Base_Quote_Generator.h5')

In [5]:
generator = keras.Sequential([
    keras.layers.InputLayer(input_shape=(cp.maxlen, len(cp.chars))),
    layers.LSTM(128),
    layers.Dense(len(cp.chars), activation="softmax")
])

In [6]:
discriminator = keras.Sequential([
    keras.layers.Conv2D(filters=40, kernel_size=(3,3), input_shape=(cp.maxlen,len(cp.chars),1), activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# train the discriminator model
def train_discriminator(model, cp, generator, n_iter=100, n_batch=128):
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_iter):
		# get randomly selected 'real' samples
		X_real, y_real = cp.generate_real_samples(half_batch)
		# update discriminator on real samples
		_, real_acc = model.train_on_batch(X_real, y_real)
		# generate 'fake' examples
		X_fake, y_fake = cp.generate_fake_samples(generator, half_batch)
		# update discriminator on fake samples
		_, fake_acc = model.train_on_batch(X_fake, y_fake)
		# summarize performance
		print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))

In [8]:
train_discriminator(discriminator, cp, generator)

NameError: name 'X_true' is not defined

In [8]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

In [ ]:
# train the composite model
def train_gan(gan_model, latent_dim, n_epochs=100, n_batch=256):
	# manually enumerate epochs
	for i in range(n_epochs):
		# prepare points in latent space as input for the generator
		x_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		gan_model.train_on_batch(x_gan, y_gan)